In [1]:
from googletrans import Translator
import numpy as np
import snorkel
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import MajorityLabelVoter
import json
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
%load_ext autoreload
%autoreload 2
import label_improve as li

In [3]:
keywords = {
    "alarm": [
        "alarm",
        "wake+up"
    ],
    "audio": [
        " mute ",
        "volume",
        " loud",
        "quiet"
    ],
    "iot": [
        "light",
        "wemo",
        "coffee"
    ],
    "calendar": [
        "calendar",
        "schedule",
        "remind"
    ],
    "play": [
        "play ",
        "podcast",
        "audiobook"
    ],
    "general": [
        "good morning",
        "joke",
        "explain"
    ],
    "datetime": [
        "date+today",
        "time+is",
        "date+is"
    ],
    "takeaway": [
        "takeaway",
        "delivery",
        "order"
    ],
    "news": [
        "news",
        "times",
        "headline"
    ],
    "music": [
        "what+song",
        "save+song",
        "shuffle"
    ],
    "weather": [
        "weather",
        "temperature",
        " rain",
        " snow"
    ],
    "qa": [
        "stock",
        "what's",
        "define",
        "describe",
        "what is",
        "what+mean"
    ],
    "social": [
        "message",
        "tweet",
        "twitter",
        "facebook",
        "complain",
        "status"
    ],
    "recommendation": [
        "recommend",
        "suggest",
        "restaurant",
    ],
    "cooking": [
        "recipe",
        "timer",
        "cook"
    ],
    "transport": [
        "ticket",
        "train",
        "flight",
        "accident",
        "traffic"
    ],
    "email": [
        "email",
        "inbox",
        "message+inbox",
        "message+email"
    ],
    "lists": [
        " list",
        "create+list",
        "delete+list"
    ]
}

In [5]:
dataset_name = "massive-ja"

idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}

ja_valid_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
ja_train_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
ja_test_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

In [6]:
# Loading the data 
dataset_name = "massive-EN"

idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
valid_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
train_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
test_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

In [7]:
keywords_by_idx = {int(label_to_idx[k]):v for k,v in keywords.items()}
lfs = li.keywords_to_LFs(keywords_by_idx)

In [8]:
new_test = li.df_to_df_with_new_lf(ja_test_df, test_df, lfs)
new_train = li.df_to_df_with_new_lf(ja_train_df, train_df, lfs)
new_valid = li.df_to_df_with_new_lf(ja_valid_df, valid_df, lfs)

100%|██████████| 3305/3305 [00:01<00:00, 2997.04it/s]


In [9]:
li.analysis_LFs_with_weak_labels(new_test, 18)
li.analysis_LFs_with_weak_labels(new_train, 18)
li.analysis_LFs_with_weak_labels(new_valid, 18)

Test Coverage: 0.610774818401937
acuracy for the not abstains
0.9107929515418502
acuracy for all
0.5006053268765133
Test Coverage: 0.6162227602905569
acuracy for the not abstains
0.901840490797546
acuracy for all
0.4957627118644068
Test Coverage: 0.6084720121028744
acuracy for the not abstains
0.9073972602739726
acuracy for all
0.5010590015128593


In [10]:
massive = li.df_to_massive(new_train)
li.save_dataset(massive,  "../weak_datasets/massive-JA/train.json")
massive = li.df_to_massive(new_valid)
li.save_dataset(massive,  "../weak_datasets/massive-JA/valid.json")
massive = li.df_to_massive(new_test)
li.save_dataset(massive,  "../weak_datasets/massive-JA/test.json")

In [11]:
# Loading the data 
dataset_name = "massive-JA"
# the following nb i actually ja for simplicity
idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
nb_valid_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
nb_train_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
nb_test_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

In [12]:
auth_key = "8648c5d9-09cf-4d89-83d5-bde6ebbc5f5e:fx"

In [13]:
nbten_test= li.deep_translate_df_to_english(nb_test_df, auth_key, "JA")
display(nbten_test.head())
display(nbten_test.tail())
nbten_test = li.massive_df_with_new_lf(nbten_test, lfs)
massive = li.df_to_massive(nbten_test)
li.save_dataset(massive, "../weak_datasets/temp/translate_ja/test.json")

Translating: 100%|██████████| 1652/1652 [00:04<00:00, 374.08it/s]


,text,label,weak_labels
0,Turn on the light,2,"[-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1..."
1,Check the meaning of pda,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,How long until 8 p.m.?,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,Did you get the email with the track number?,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,What does the email my mother sent me thirty m...,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


,text,label,weak_labels
1647,We'll get this resolved quickly.,12,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1648,Where is Laos?,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1649,Tell me if I should take the train or the bus ...,15,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1650,Tell me about Selena Gomez's family.,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1651,The date of the winter solstice two years from...,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


100%|██████████| 1652/1652 [00:00<00:00, 3694.42it/s]


In [14]:
nbten_valid= li.deep_translate_df_to_english(nb_valid_df, auth_key, "JA")
display(nbten_valid.head())
display(nbten_valid.tail())
nbten_valid = li.massive_df_with_new_lf(nbten_valid, lfs)
massive = li.df_to_massive(nbten_valid)
li.save_dataset(massive, "../weak_datasets/temp/translate_ja/valid.json")

Translating: 100%|██████████| 3305/3305 [00:06<00:00, 487.35it/s]


,text,label,weak_labels
0,Look up narendra modi.,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,What is the weather forecast for this week,10,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,Email Yu to wish him a Happy New Year.,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,Tell me about a furniture store near you.,13,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,Turn off the bedroom light.,2,"[-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1..."


,text,label,weak_labels
3300,Play music.,4,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3301,The conversion rate between dirhams and rupees is,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3302,When is Friday the 13th this year?,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3303,Complete all events in the calendar,3,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1..."
3304,Say hello and post on facebook.,12,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


100%|██████████| 3305/3305 [00:01<00:00, 3105.35it/s]


In [15]:
nbten_train= li.deep_translate_df_to_english(nb_train_df, auth_key, "JA")
display(nbten_train.head())
display(nbten_train.tail())
nbten_train = li.massive_df_with_new_lf(nbten_train, lfs)
massive = li.df_to_massive(nbten_train)
li.save_dataset(massive, "../weak_datasets/temp/translate_ja/train.json")

Translating: 100%|██████████| 11564/11564 [00:20<00:00, 564.78it/s]


,text,label,weak_labels
0,Open my list,17,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,Disable Shuffle,9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,What listings have you created today?,17,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,I bumped into my old friend no Kosuke today.,5,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,Busy day.,5,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


,text,label,weak_labels
11559,restaurant (esp. Western-style),13,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11560,I'll call a cab today at 7pm.,15,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11561,Play Paprika by Genji Yonezu.,4,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11562,Turn off the 6 a.m. alarm on Wednesdays,0,"[0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1..."
11563,Text Kotaro to ask what time dinner is.,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


100%|██████████| 11564/11564 [00:03<00:00, 2990.60it/s]


In [16]:
# Loading the data 
dataset_name = "massive-JA"

idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
nb_valid_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
nb_train_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
nb_test_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

In [17]:
# Loading the data 
dataset_name = "translate_ja"

nte_valid_df = li.massive_to_df(json.load(open(f"../weak_datasets/temp/{dataset_name}/valid.json", "r")))
nte_train_df = li.massive_to_df(json.load(open(f"../weak_datasets/temp/{dataset_name}/train.json", "r")))
nte_test_df = li.massive_to_df(json.load(open(f"../weak_datasets/temp/{dataset_name}/test.json", "r")))

In [18]:
new_test = li.df_to_df_with_new_lf(nb_test_df, nte_test_df, lfs)
massive = li.df_to_massive(new_test)
li.save_dataset(massive,  "../weak_datasets/massive-JA2/test.json")
new_train = li.df_to_df_with_new_lf(nb_train_df, nte_train_df, lfs)
massive = li.df_to_massive(new_train)
li.save_dataset(li.df_to_massive(new_train),  "../weak_datasets/massive-JA2/train.json")
new_valid = li.df_to_df_with_new_lf(nb_valid_df, nte_valid_df, lfs)
massive = li.df_to_massive(new_valid)
li.save_dataset(li.df_to_massive(new_valid),  "../weak_datasets/massive-JA2/valid.json")


100%|██████████| 3305/3305 [00:01<00:00, 3288.62it/s]


In [19]:
li.analysis_LFs_with_weak_labels(new_valid,18)  
li.analysis_LFs_with_weak_labels(new_train,18)
li.analysis_LFs_with_weak_labels(new_test,18)

Test Coverage: 0.5709531013615734
acuracy for the not abstains
0.9081395348837209
acuracy for all
0.4726172465960666
Test Coverage: 0.5763576617087512
acuracy for the not abstains
0.9098226018396847
acuracy for all
0.4789865098581806
Test Coverage: 0.5641646489104116
acuracy for the not abstains
0.926056338028169
acuracy for all
0.47760290556900725


In [20]:
li.analysis_LFs_with_weak_labels(valid_df,18)
li.analysis_LFs_with_weak_labels(train_df,18)
li.analysis_LFs_with_weak_labels(test_df,18)

Test Coverage: 0.6084720121028744
acuracy for the not abstains
0.9073972602739726
acuracy for all
0.5010590015128593
Test Coverage: 0.6162227602905569
acuracy for the not abstains
0.901840490797546
acuracy for all
0.4957627118644068
Test Coverage: 0.610774818401937
acuracy for the not abstains
0.9107929515418502
acuracy for all
0.5006053268765133


In [6]:
display(train_df.head())
display(train_df.tail())
display(valid_df.head())
display(valid_df.tail())
display(test_df.head())
display(test_df.tail())

,text,label,weak_labels
0,open my list,17,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,disable shuffle,9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,what list did i make today,17,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,today i ran into my old friend bob,5,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,it was a work load day,5,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


,text,label,weak_labels
11559,guide me to the restaurant,13,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11560,call golden taxi to pick me up at seven p. m. ...,15,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11561,assistant please play royksopp running to the sea,4,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11562,turn off the six am alarm for wednesday,0,"[0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1..."
11563,send email to robert what time is dinner,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


,text,label,weak_labels
0,tell me about narendra modi,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,what is the weather forecast for this week,10,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,email happy new year to john,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,can you find me a furniture store near me,13,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,turn my bedroom lights off,2,"[-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1..."


,text,label,weak_labels
3300,play the music,4,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3301,exchange rate dirham and rupees,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3302,which day is friday the thirteenth this year,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3303,clear all my calendar events,3,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1..."
3304,post hey everyone on facebook,12,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


,text,label,weak_labels
0,can you turn up the lights,2,"[-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1..."
1,what does pda mean,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,how close is it to eight p. m.,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,have i received any emails with tracking numbers,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,what does the email say that was sent thirty m...,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


,text,label,weak_labels
1647,get it fast resolved,12,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1648,where is laos located,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1649,tell me if i should use the train or bus to ge...,15,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1650,tell me background family of selena gomez,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1651,what's the date of the winter solstice two yea...,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


In [7]:
li.analysis_LFs_with_weak_labels(train_df,18)
li.analysis_LFs_with_weak_labels(valid_df,18)
li.analysis_LFs_with_weak_labels(test_df,18)

Test Coverage: 0.6162227602905569
acuracy for the not abstains
0.901840490797546
acuracy for all
0.4957627118644068
Test Coverage: 0.6084720121028744
acuracy for the not abstains
0.9073972602739726
acuracy for all
0.5010590015128593
Test Coverage: 0.610774818401937
acuracy for the not abstains
0.9107929515418502
acuracy for all
0.5006053268765133


In [8]:
# Loading the data 
dataset_name = "massive-CN"

idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
cn_valid_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
cn_train_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
cn_test_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

In [9]:
display(cn_train_df.head())
display(cn_train_df.tail())
display(cn_valid_df.head())
display(cn_valid_df.tail())
display(cn_test_df.head())
display(cn_test_df.tail())

,text,label,weak_labels
0,打开我的清单,17,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,关闭随机播放,9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,我今天有什么清单吗,17,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,今天我偶然碰见老朋友大强了,5,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,是一个忙碌的一天,5,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


,text,label,weak_labels
11559,为我导航去餐馆,13,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11560,今天晚上七点叫滴滴来接我,15,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11561,助理请播放许嵩的断桥残雪,4,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11562,关停周三六点的闹钟,0,"[0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1..."
11563,发邮件给家俊什么时间吃晚餐,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


,text,label,weak_labels
0,告诉我关于纳伦德拉莫迪,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,这周天气预报什么情况啊,10,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,给约翰发送电子邮件说新年快乐,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,你能在附近帮我找一家家具店吗,13,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,关闭我的卧室灯光,2,"[-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1..."


,text,label,weak_labels
3300,播音乐,4,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3301,迪拉姆和卢比的汇率,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3302,今年哪一天是十三号星期五,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3303,清除我的所有的日历活动,3,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1..."
3304,发微博说哈喽大家好,12,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


,text,label,weak_labels
0,你能把灯打开吗,2,"[-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1..."
1,个人数字助理是什么意思,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,和下午八点还差多少,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,我有收到带有追踪号码的邮件吗,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,妈妈三十分钟前发的电子邮件说了什么,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


,text,label,weak_labels
1647,让它尽快解决,12,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1648,老挝位于哪里,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1649,告诉我今天乘火车还是乘公共汽车穿过市区,15,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1650,告诉我王菲的家庭背景,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1651,两年后的冬至是几月几号,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


In [10]:
auth_key = ""

In [32]:
cnten_test = li.deep_translate_df_to_english(cn_test_df, auth_key, "ZH")

Translating: 100%|██████████| 1652/1652 [00:03<00:00, 444.75it/s]


In [34]:
cnten_test = li.massive_df_with_new_lf(cnten_test, lfs)

100%|██████████| 1652/1652 [00:00<00:00, 3806.34it/s]


In [36]:
massive = li.df_to_massive(cnten_test)
li.save_dataset(massive, "../weak_datasets/temp/translate_cn/test.json")

In [11]:
cnten_train= li.deep_translate_df_to_english(cn_train_df, auth_key, "ZH")
display(cnten_train.head())
display(cnten_train.tail())
cnten_train = li.massive_df_with_new_lf(cnten_train, lfs)
massive = li.df_to_massive(cnten_train)
li.save_dataset(massive, "../weak_datasets/temp/translate_cn/train.json")

Translating: 100%|██████████| 11564/11564 [00:21<00:00, 531.02it/s]


,text,label,weak_labels
0,Open my list,17,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,Turn off randomized playback,9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,Do I have a list for today?,17,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,"I ran into my old friend, Keung, today.",5,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,Was a busy day,5,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


,text,label,weak_labels
11559,Navigate me to the restaurant.,13,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11560,I'll be picked up at 7:00 p.m. today.,15,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11561,"Assistant, please play ""Broken Bridge and Brok...",4,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11562,Turn off the 6:00 alarm on Wednesdays.,0,"[0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1..."
11563,Email Ka-chun what time he's having dinner.,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


100%|██████████| 11564/11564 [00:03<00:00, 3728.71it/s]


In [12]:
cnten_valid= li.deep_translate_df_to_english(cn_valid_df, auth_key, "ZH")
display(cnten_valid.head())
display(cnten_valid.tail())
cnten_valid = li.massive_df_with_new_lf(cnten_valid, lfs)
massive = li.df_to_massive(cnten_valid)
li.save_dataset(massive, "../weak_datasets/temp/translate_cn/valid.json")

Translating: 100%|██████████| 3305/3305 [00:06<00:00, 488.67it/s]


,text,label,weak_labels
0,Tell me about Narendra Modi.,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,What's the weather forecast for this week?,10,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,Email John to say Happy New Year!,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,Can you find me a furniture store in the neigh...,13,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,Turn off my bedroom lights.,2,"[-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1..."


,text,label,weak_labels
3300,broadcast music,4,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3301,Exchange rate of dirhams and rupees,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3302,What day is Friday the 13th this year?,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3303,Clear all my calendar events,3,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1..."
3304,Tweets hello to everyone.,12,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


100%|██████████| 3305/3305 [00:00<00:00, 3719.59it/s]


In [13]:
li.analysis_LFs_with_weak_labels(cnten_valid,18)

Test Coverage: 0.5579425113464448
acuracy for the not abstains
0.8862418106015485
acuracy for all
0.4502269288956127


In [15]:
# Loading the data 
dataset_name = "massive-NB"

idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
nb_valid_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
nb_train_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
nb_test_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

In [16]:
nbten_train= li.deep_translate_df_to_english(nb_train_df, auth_key, "NB")
display(nbten_train.head())
display(nbten_train.tail())
nbten_train = li.massive_df_with_new_lf(nbten_train, lfs)
massive = li.df_to_massive(nbten_train)
li.save_dataset(massive, "../weak_datasets/temp/translate_nb/train.json")

Translating: 100%|██████████| 11564/11564 [00:38<00:00, 300.89it/s]


,text,label,weak_labels
0,open my list,17,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,turn off random order,9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,which list did I create today,17,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,Today I met my old friend per,5,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,it was a day with a lot of work,5,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


,text,label,weak_labels
11559,guide me to the restaurant,13,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11560,call norgestaxi to pick me up at nineteen today,15,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11561,assistant please play röyksopp running to the sea,4,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
11562,turn off the alarm at six o'clock on Wednesday,0,"[0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1..."
11563,send email to robert when is dinner,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


100%|██████████| 11564/11564 [00:03<00:00, 3748.30it/s]


In [17]:
nbten_test= li.deep_translate_df_to_english(nb_test_df, auth_key, "NB")
display(nbten_test.head())
display(nbten_test.tail())
nbten_test = li.massive_df_with_new_lf(nbten_test, lfs)
massive = li.df_to_massive(nbten_test)
li.save_dataset(massive, "../weak_datasets/temp/translate_nb/test.json")

Translating: 100%|██████████| 1652/1652 [00:04<00:00, 398.38it/s]


,text,label,weak_labels
0,you can turn up lights,2,"[-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1..."
1,what does p. d. a. mean?,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,how long is it until 8 p.m.,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,I have received some emails with tracking numbers,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,what does it say in the email sent thirty minu...,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


,text,label,weak_labels
1647,get it resolved quickly,12,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1648,where is laos located,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1649,tell me if I should use the train or bus to ge...,15,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1650,tell me about tone damli's family,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1651,what date is the winter solstice in two years?,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


100%|██████████| 1652/1652 [00:00<00:00, 3637.92it/s]


In [18]:
nbten_valid= li.deep_translate_df_to_english(nb_valid_df, auth_key, "NB")
display(nbten_valid.head())
display(nbten_valid.tail())
nbten_valid = li.massive_df_with_new_lf(nbten_valid, lfs)
massive = li.df_to_massive(nbten_valid)
li.save_dataset(massive, "../weak_datasets/temp/translate_nb/valid.json")

Translating: 100%|██████████| 3305/3305 [00:07<00:00, 416.23it/s]


,text,label,weak_labels
0,tell me about une bastholm,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,what is the weather forecast for this week,10,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,e -mail happy new year to john,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,can you find me a furniture store near me,13,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,Turn my bedroom lights off,2,"[-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1..."


,text,label,weak_labels
3300,play the music,4,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3301,exchange rate dirham and rupees,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3302,what day is friday the thirteenth this year,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3303,remove all my events in the calendar,3,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 3, -1, -1..."
3304,post hello everyone on facebook,12,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


100%|██████████| 3305/3305 [00:00<00:00, 3727.45it/s]


In [19]:
# Loading the data 
dataset_name = "massive-CN"

idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
cn_valid_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
cn_train_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
cn_test_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

In [20]:
# Loading the data 
dataset_name = "translate_cn"

cte_valid_df = li.massive_to_df(json.load(open(f"../weak_datasets/temp/{dataset_name}/valid.json", "r")))
cte_train_df = li.massive_to_df(json.load(open(f"../weak_datasets/temp/{dataset_name}/train.json", "r")))
cte_test_df = li.massive_to_df(json.load(open(f"../weak_datasets/temp/{dataset_name}/test.json", "r")))

In [21]:
new_test = li.df_to_df_with_new_lf(cn_test_df, cte_test_df, lfs)
massive = li.df_to_massive(new_test)
li.save_dataset(massive,  "../weak_datasets/massive-CN2/test.json")
new_train = li.df_to_df_with_new_lf(cn_train_df, cte_train_df, lfs)
massive = li.df_to_massive(new_train)
li.save_dataset(li.df_to_massive(new_train),  "../weak_datasets/massive-CN2/train.json")
new_valid = li.df_to_df_with_new_lf(cn_valid_df, cte_valid_df, lfs)
massive = li.df_to_massive(new_valid)
li.save_dataset(li.df_to_massive(new_valid),  "../weak_datasets/massive-CN2/valid.json")


100%|██████████| 3305/3305 [00:00<00:00, 3691.50it/s]


In [22]:
li.analysis_LFs_with_weak_labels(new_valid,18)  
li.analysis_LFs_with_weak_labels(new_train,18)
li.analysis_LFs_with_weak_labels(new_test,18)

Test Coverage: 0.5579425113464448
acuracy for the not abstains
0.8862418106015485
acuracy for all
0.4502269288956127
Test Coverage: 0.5627810446212383
acuracy for the not abstains
0.8833873996926754
acuracy for all
0.4474230370114147
Test Coverage: 0.5423728813559322
acuracy for the not abstains
0.8894348894348895
acuracy for all
0.43825665859564167


In [25]:
# Loading the data 
dataset_name = "massive-NB"

idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
nb_valid_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
nb_train_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
nb_test_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

In [26]:
# Loading the data 
dataset_name = "translate_nb"

nte_valid_df = li.massive_to_df(json.load(open(f"../weak_datasets/temp/{dataset_name}/valid.json", "r")))
nte_train_df = li.massive_to_df(json.load(open(f"../weak_datasets/temp/{dataset_name}/train.json", "r")))
nte_test_df = li.massive_to_df(json.load(open(f"../weak_datasets/temp/{dataset_name}/test.json", "r")))

In [27]:
new_test = li.df_to_df_with_new_lf(nb_test_df, nte_test_df, lfs)
massive = li.df_to_massive(new_test)
li.save_dataset(massive,  "../weak_datasets/massive-NB2/test.json")
new_train = li.df_to_df_with_new_lf(nb_train_df, nte_train_df, lfs)
massive = li.df_to_massive(new_train)
li.save_dataset(li.df_to_massive(new_train),  "../weak_datasets/massive-NB2/train.json")
new_valid = li.df_to_df_with_new_lf(nb_valid_df, nte_valid_df, lfs)
massive = li.df_to_massive(new_valid)
li.save_dataset(li.df_to_massive(new_valid),  "../weak_datasets/massive-NB2/valid.json")


100%|██████████| 3305/3305 [00:00<00:00, 3436.21it/s]


In [28]:
li.analysis_LFs_with_weak_labels(new_valid,18)  
li.analysis_LFs_with_weak_labels(new_train,18)
li.analysis_LFs_with_weak_labels(new_test,18)

Test Coverage: 0.5897125567322239
acuracy for the not abstains
0.9057239057239057
acuracy for all
0.48835098335854765
Test Coverage: 0.5916637841577309
acuracy for the not abstains
0.9011787502018408
acuracy for all
0.4826184711172605
Test Coverage: 0.5901937046004843
acuracy for the not abstains
0.9082774049217002
acuracy for all
0.4915254237288136


In [29]:
li.analysis_LFs_with_weak_labels(valid_df,18)
li.analysis_LFs_with_weak_labels(train_df,18)
li.analysis_LFs_with_weak_labels(test_df,18)

Test Coverage: 0.6084720121028744
acuracy for the not abstains
0.9073972602739726
acuracy for all
0.5010590015128593
Test Coverage: 0.6162227602905569
acuracy for the not abstains
0.901840490797546
acuracy for all
0.4957627118644068
Test Coverage: 0.610774818401937
acuracy for the not abstains
0.9107929515418502
acuracy for all
0.5006053268765133


# Save datasets

# Chinese setup


In [5]:
dataset_name = "massive-cn"

idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
cn_valid_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
cn_train_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
cn_test_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

In [16]:
new_test = li.df_to_df_with_new_lf(cn_test_df, test_df, lfs)
new_train = li.df_to_df_with_new_lf(cn_train_df, train_df, lfs)
new_valid = li.df_to_df_with_new_lf(cn_valid_df, valid_df, lfs)

  0%|          | 0/1652 [00:00<?, ?it/s]

100%|██████████| 3305/3305 [00:00<00:00, 3535.54it/s]


In [17]:
new_test.head()

,text,label,weak_labels
0,你能把灯打开吗,2,"[-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1..."
1,个人数字助理是什么意思,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,和下午八点还差多少,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,我有收到带有追踪号码的邮件吗,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,妈妈三十分钟前发的电子邮件说了什么,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


In [18]:
massive = li.df_to_massive(new_train)
li.save_dataset(massive,  "../weak_datasets/massive-CN/train.json")
massive = li.df_to_massive(new_valid)
li.save_dataset(massive,  "../weak_datasets/massive-CN/valid.json")
massive = li.df_to_massive(new_test)
li.save_dataset(massive,  "../weak_datasets/massive-CN/test.json")

# Start translate version

In [19]:
cn_train_df.head()

,text,label,weak_labels
0,打开我的清单,17,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,关闭随机播放,9,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,我今天有什么清单吗,17,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,今天我偶然碰见老朋友大强了,5,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,是一个忙碌的一天,5,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


In [7]:
# translate all the data
cn_en_test = li.translate_df_to_english(cn_test_df, 'zh-CN')
new_test = li.df_to_df_with_new_lf(cn_test_df, cn_en_test, lfs)
massive = li.df_to_massive(new_test)
li.save_dataset(massive,  "../weak_datasets/massive-CN2/test.json")

Translating:   0%|          | 0/67 [00:00<?, ?it/s]

Translating:   1%|▏         | 1/67 [00:33<36:29, 33.17s/it]


KeyboardInterrupt: 

In [8]:
# Example usage
auth_key = "8648c5d9-09cf-4d89-83d5-bde6ebbc5f5e:fx"
# translate all the data
cn_en_test = li.deep_translate_df_to_english(cn_test_df, auth_key, 'ZH')
new_test = li.massive_df_with_new_lf(cn_test_df, lfs)
massive = li.df_to_massive(new_test)
li.save_dataset(massive, "../weak_datasets/temp/translate_cn/test.json")

Translating: 100%|██████████| 1652/1652 [00:03<00:00, 450.75it/s]


NameError: name 'lfs' is not defined

In [11]:
new_test = li.massive_df_with_new_lf(cn_en_test, lfs)
massive = li.df_to_massive(new_test)
li.save_dataset(massive, "../weak_datasets/temp/translate_cn/test.json")

  0%|          | 0/1652 [00:00<?, ?it/s]

100%|██████████| 1652/1652 [00:00<00:00, 3630.24it/s]


In [22]:
cn_en_test.head()


,text,label,weak_labels
0,Can you turn on the light?,2,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
1,Personal digital assistant What does it mean,11,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2,How much worse than 8 pm,6,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,Have I received an email with a tracking number?,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4,What did my mother say 30 minutes ago?,16,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."


In [27]:
# translate all the data
cn_en_valid = li.translate_df_to_english(cn_valid_df, 'zh-CN')
new_valid = li.df_to_df_with_new_lf(cn_valid_df, cn_en_valid, lfs)
massive = li.df_to_massive(new_valid)
li.save_dataset(massive,  "../weak_datasets/massive-CN2/valid.json")

Translating:   0%|          | 0/133 [00:00<?, ?it/s]

100%|██████████| 3305/3305 [00:00<00:00, 3464.07it/s]


In [32]:
# Example usage
auth_key = ""
# translate all the data
cn_en_train = li.deep_translate_df_to_english(cn_train_df, auth_key, 'ZH')
new_train = li.df_to_df_with_new_lf(cn_train_df, cn_en_train, lfs)
massive = li.df_to_massive(new_train)
li.save_dataset(massive,  "../weak_datasets/massive-CN2/train.json")

Translating:   0%|          | 0/11564 [00:00<?, ?it/s]

100%|██████████| 11564/11564 [00:03<00:00, 3439.00it/s]


In [34]:

massive = li.df_to_massive(cn_en_train)
li.save_dataset(massive,  "./translated_train.json")

In [35]:
cn_en_train.head(19)
new_train = li.massive_df_with_new_lf(cn_en_train, lfs)
massive = li.df_to_massive(new_train)
li.save_dataset(massive,  "./translated_train.json")


  0%|          | 0/11564 [00:00<?, ?it/s]

100%|██████████| 11564/11564 [00:03<00:00, 3666.90it/s]


In [36]:
lf_analysis = li.analysis_LFs(lfs, new_train, 18)

100%|██████████| 11564/11564 [00:03<00:00, 3693.72it/s]


Test Coverage: 0.562953995157385
acuracy for the not abstains
0.36666097935505887
acuracy for all
0.18583535108958837


In [40]:
lf_analysis = li.analysis_LFs_with_weak_labels(new_train, 18)

Test Coverage: 0.562953995157385
acuracy for the not abstains
0.36666097935505887
acuracy for all
0.18583535108958837


In [42]:
cn_train = li.massive_to_df(json.load(open(f"../weak_datasets/massive-CN/train.json", "r")))

In [43]:
lf_analysis = li.analysis_LFs_with_weak_labels(cn_train, 18)

Test Coverage: 0.6162227602905569
acuracy for the not abstains
0.901840490797546
acuracy for all
0.4957627118644068


## Norwegian setup 

In [12]:
dataset_name = "massive-nb"

idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}

nb_valid_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
nb_train_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
nb_test_df = li.massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

In [56]:
new_test = li.df_to_df_with_new_lf(nb_test_df, test_df, lfs)
new_train = li.df_to_df_with_new_lf(nb_train_df, train_df, lfs)
new_valid = li.df_to_df_with_new_lf(nb_valid_df, valid_df, lfs)

100%|██████████| 3305/3305 [00:00<00:00, 3697.77it/s]


In [57]:
massive = li.df_to_massive(new_train)
li.save_dataset(massive,  "../weak_datasets/massive-NB/train.json")
massive = li.df_to_massive(new_valid)
li.save_dataset(massive,  "../weak_datasets/massive-NB/valid.json")
massive = li.df_to_massive(new_test)
li.save_dataset(massive,  "../weak_datasets/massive-NB/test.json")

In [58]:
li.analysis_LFs_with_weak_labels(new_train, 18)
li.analysis_LFs_with_weak_labels(new_valid, 18)
li.analysis_LFs_with_weak_labels(new_test, 18)

Test Coverage: 0.6162227602905569
acuracy for the not abstains
0.901840490797546
acuracy for all
0.4957627118644068
Test Coverage: 0.6084720121028744
acuracy for the not abstains
0.9073972602739726
acuracy for all
0.5010590015128593
Test Coverage: 0.610774818401937
acuracy for the not abstains
0.9107929515418502
acuracy for all
0.5006053268765133


In [59]:
# Example usage
auth_key = "82b8da97-a9d0-474e-9002-bb9d776d7c9c:fx"
# translate all the data
nb_en_test = li.deep_translate_df_to_english(nb_test_df, auth_key, 'NB')
new_test = li.df_to_df_with_new_lf(nb_test_df, nb_en_test, lfs)
massive = li.df_to_massive(new_test)
li.save_dataset(massive,  "../weak_datasets/massive-NB2/test.json")

100%|██████████| 1652/1652 [00:00<00:00, 3662.25it/s]


In [61]:
lf_analysis = li.analysis_LFs_with_weak_labels(new_test, 18)

Test Coverage: 0.5901937046004843
acuracy for the not abstains
0.3691275167785235
acuracy for all
0.19975786924939468


In [63]:
# Example usage
auth_key = "8e406fcb-2e39-41a9-8c7e-05668042bf46:fx"
# translate all the data
nb_en_train = li.deep_translate_df_to_english(nb_train_df, auth_key, 'NB')
new_train = li.df_to_df_with_new_lf(nb_train_df, nb_en_train, lfs)
massive = li.df_to_massive(new_train)
li.save_dataset(massive,  "../weak_datasets/massive-NB2/train.json")

Translating:   0%|          | 0/11564 [00:00<?, ?it/s]

100%|██████████| 11564/11564 [00:03<00:00, 3627.39it/s]


In [64]:
lf_analysis = li.analysis_LFs_with_weak_labels(new_train, 18)

Test Coverage: 0.5915773088896575
acuracy for the not abstains
0.35142118863049093
acuracy for all
0.18817018332756832


In [66]:
# Example usage
auth_key = ""
# translate all the data
nb_en_valid = li.deep_translate_df_to_english(nb_valid_df, auth_key, 'NB')
new_valid = li.df_to_df_with_new_lf(nb_valid_df, nb_en_valid, lfs)
massive = li.df_to_massive(new_valid)
li.save_dataset(massive,  "../weak_datasets/massive-NB2/valid.json")

Translating:   0%|          | 0/3305 [00:00<?, ?it/s]

100%|██████████| 3305/3305 [00:00<00:00, 3667.86it/s]


In [67]:
lf_analysis = li.analysis_LFs_with_weak_labels(new_valid, 18)

Test Coverage: 0.5897125567322239
acuracy for the not abstains
0.40404040404040403
acuracy for all
0.2178517397881997


In [68]:
new_train = li.massive_df_with_new_lf(nb_en_train, lfs)
massive = li.df_to_massive(new_train)
li.save_dataset(massive,  "../weak_datasets/temp/translate_nb/train.json")

100%|██████████| 11564/11564 [00:03<00:00, 3632.39it/s]


In [69]:
new_test = li.massive_df_with_new_lf(nb_en_test, lfs)
massive = li.df_to_massive(new_test)
li.save_dataset(massive,  "../weak_datasets/temp/translate_nb/test.json")

100%|██████████| 1652/1652 [00:00<00:00, 3712.34it/s]


In [70]:
new_valid = li.massive_df_with_new_lf(nb_en_valid, lfs)
massive = li.df_to_massive(new_valid)
li.save_dataset(massive,  "../weak_datasets/temp/translate_nb/valid.json")

100%|██████████| 3305/3305 [00:00<00:00, 3710.79it/s]
